In [27]:
# from Gapfilling import *
from modelseed_gapfilling import *
import re
import gurobipy
import cobra
from cobra.flux_analysis.gapfilling import GapFiller
from cobra.io import read_sbml_model
from cobra import exceptions
from contextlib import contextmanager
import sys, os
import pandas as pd

In [28]:
def load_query_model(model, obj = None):
    """
    Loads a model. Objective can be changed to either "biomass" (regex
    will be used) or a valid reaction ID.
    Print is used temporary. A general log will be generated in the 
    final version.
    """
    Model = model
    if obj == "biomass":
        b = re.compile("biomass", re.IGNORECASE)
        bc = re.compile("(biomass){1}.*(core){1}", re.IGNORECASE)
        reactions = [reaction.id for reaction in Model.reactions]
        # searching for a biomass core reaction
        core = list(filter(bc.match, reactions))
        if core:
            Model.objective = core[0]
            return Model
        # searching for a non core biomass reaction
        biomass = list(filter(b.match, reactions))
        if biomass:
            Model.objective = biomass[0]
            return Model
        print("biomass not found in the query model")
        return Model
    if obj != "biomass" and obj != None:
        try:
            Model.objective = obj
        except ValueError:
            print(str(obj) + " not found in the query model")
            return Model
        return Model
    if obj == None:
        return Model

def load_template_models(template_list, obj = None):
    """
    Takes a list of template models and changes objective if specified.
    Objective can be either "biomass" or a specific reaction ID.
    """
    templates = []
    for t in template_list:
        templates.append(t)
    failures = []
    if obj == None:
        return templates
    if obj == "biomass":
        b = re.compile("biomass", re.IGNORECASE)
        bc = re.compile("(biomass){1}.*(core){1}", re.IGNORECASE) 
        for model in templates:
            reactions = [reaction.id for reaction in model.reactions]
            # Searching for a biomass_core reaction
            core = list(filter(bc.match, reactions))
            if core:
                model.objective = core[0]
                continue
            # Searching for a non core biomass reaction
            biomass = list(filter(b.match, reactions))
            if biomass:
                model.objective = biomass[0]
            # If biomass reactions are not found, the model name is stored into "failures" list.
            # The model won't change its objective, but it will be used anyway for gap filling
            else:
                failures.append(str(model))
            print(failures)
            return templates
    if obj != None and obj != "biomass":
        for model in templates:
            try:
                model.objective = obj
            except ValueError:
                failures.append(str(model))
        print(failures)
        return templates
    
def gapfilling(model, template, integer_threshold, it = 1):
    """
    Calls GapFiller class from cobrapy.
    """
    gapfiller = GapFiller(model, template, integer_threshold = integer_threshold,
                         demand_reactions = False)
    return gapfiller.fill(iterations = it)

def add_exchange_reactions(model, template):
    """
    Adds all exchange reactions from a template matching model metabolites. 
    """
    EX = [template.reactions.get_by_id(i.id) for i in template.reactions if i.id.startswith("EX_")]
    for reaction in EX:
        if reaction not in model.reactions and str(list(reaction.metabolites.keys())[0]) in [i.id for i in model.metabolites]:
            model.add_reaction(reaction.copy())
    return model

def is_transport(reaction, compartments_list, all_compounds = False, ignore_h = False):
    """
    Takes a cobra model reaction as input and determines if it is a transport reaction.
    all_compounds = True -> both sides of the reaction must be the same
    all_compounds = False -> at least one compound must be in both sides
    ignore_h = True -> does not consider proton transference as transport
    """
    # left part of the reaction
    r = list(str(x) for x in reaction.reactants)
    # right part
    p = list(str(x) for x in reaction.products)
    if len(r) != len(p):
        return False
    c = compartments_list
    # removing terminations
    R = []
    for i in r:
        for e in c:
            i = i.replace(e, "") 
        R.append(i)
    P = []
    for i in p:
        for e in c:
            i = i.replace(e, "")
        P.append(i)
    # we sort the lists to avoid missing transport reactions where the sequence of the compounds is not maintained
    R = sorted(R)
    P = sorted(P)
    if all_compounds == False:
        if ignore_h == True:
            if "h" in R:
                R.remove("h")
            if "h" in P:
                P.remove("h")
        for i in R:
            if i in P:
                return True
            else:
                return False
    if R == P:
        return True
    else:
        return False
    
def add_transport(model, template, all_compounds = False, ignore_h = False):
    """
    Adds transport reactions from a template which metabolites are present in the model.
    """
    # PREPARATION
    # template compartments
    t_compartments = list(template.compartments.keys())
    t_Compartments = []
    for i in t_compartments:
        t_Compartments.append("_" + str(i))
    # getting compartments and metabolites from query model for further use
    m_compartments = list(model.compartments.keys())
    m_Compartments = []
    for i in m_compartments:
        m_Compartments.append("_" + str(i))
    m_metabolites = list(str(x) for x in model.metabolites)
    # removing suffixes from metabolites
    m_Metabolites = []
    for i in m_metabolites:
        for e in m_Compartments:
            i = i.replace(e, "")
        m_Metabolites.append(i)
    # sorting and removing duplicates
    m_Metabolites = list(dict.fromkeys(m_Metabolites))
    # REACTION ADDING
    for reaction in template.reactions:
        if is_transport(reaction, t_Compartments, all_compounds = all_compounds, ignore_h = ignore_h) and reaction not in model.reactions:
            # we will only use reactants as they'll be the same as products (apart from location)
            m = list(str(x) for x in reaction.reactants)
            M = []
            for i in m:
                for e in t_Compartments:
                    i = i.replace(e, "")
                M.append(i)
            if all(x in m_Metabolites for x in M):  
                model.add_reaction(reaction.copy())
    return model

def homology_gapfilling(model, templates, model_obj = None, template_obj = None, use_all_templates = False,
                       integer_threshold = 1e-6, force_exchange = False, force_transport = False, t_all_compounds = False,
                       t_ignore_h = False, value_fraction = 0.8):
    """
    Performs gap filling on a model using homology models as templates.
    """
    Model = load_query_model(model, obj = model_obj)
    Model.solver = 'gurobi'
    Templates = load_template_models(template_list = templates, obj = template_obj)
    # this dict will store used models, genes and reactions
    added_reactions = {}
    # initial flux value
    value = Model.optimize().objective_value
    if value == None:
        value = 0.0
    if use_all_templates == False:
        for template in Templates:
            # adding exchange reactions
            if force_exchange == True:
                add_exchange_reactions(Model, template)
            # adding transport reactions
            if force_transport == True:
                add_transport(Model, template, all_compounds = t_all_compounds, ignore_h = t_ignore_h)
            template.solver = 'gurobi'
            try:
                # reactions "log"
                log = []
                # result variable will store the reactions ids
                result = gapfilling(Model, template, integer_threshold = integer_threshold)
                for reaction in result[0]:
                    if reaction.id.startswith("EX_"):
                        log.append((reaction.id, "Exchange reaction"))
                    else:
                        log.append((reaction.id, [str(list(reaction.genes)[i]) 
                                              for i in range(len(list(reaction.genes)))]))
                added_reactions[str(template)] = log
                # Adding reactions to the model
                [Model.add_reaction(reaction.copy()) for reaction in result[0]]
                # Flux will be evaluated here
                new_value = Model.optimize().objective_value
                if new_value != None and new_value > value:
                    value = new_value
                elif new_value == None:
                    continue
                elif new_value != None and new_value == value:
                    break
                elif new_value < value:
                    if new_value >= value * value_fraction:
                        for i in range(len(log)):
                            Model.reactions.log[i][0].lower_bound = 0.
                            Model.reactions.log[i][0].upper_bound = 0.
                        new_value = Model.optimize().objective_value
                        value = new_value
                    else:
                        [Model.remove_reactions(Model.reactions.log[i][0]) for i in range(len(log))]
                        del added_reactions[str(template)]
                        break
                write_sbml_model(Model, filename = "model.tmp")
                Model = read_sbml_model("model.tmp")
                os.remove("model.tmp")
            except RuntimeError:
                print("\n" + str(template) + ": failed to validate gapfilled model, try lowering the integer_threshold")
            except exceptions.Infeasible:
                print("\n" + str(template) + ": gapfilling optimization failed (infeasible)")
        return Model, added_reactions
    else:
        for template in Templates:
            # adding exchange reactions
            if force_exchange == True:
                add_exchange_reactions(Model, template)
            # adding transport reactions
            if force_transport == True:
                add_transport(Model, template, all_compounds = t_all_compounds, ignore_h = t_ignore_h)
            template.solver = 'gurobi'
            try:
                log = []
                result = gapfilling(Model, template, integer_threshold = integer_threshold)   
                for reaction in result[0]:
                    if reaction.id.startswith("EX_"):
                        log.append((reaction.id, "Exchange reaction"))
                    else:
                        log.append((reaction.id, [str(list(reaction.genes)[i]) 
                                              for i in range(len(list(reaction.genes)))]))
                added_reactions[str(template)] = log
                [Model.add_reaction(reaction.copy()) for reaction in result[0]]
                write_sbml_model(Model, filename = "model.tmp")
                Model = read_sbml_model("model.tmp")
                os.remove("model.tmp")
            except RuntimeError:
                print("\n" + str(template) + ": failed to validate gapfilled model, try lowering the integer_threshold")
            except exceptions.Infeasible:
                print("\n" + str(template) + ": gapfilling optimization failed (infeasible)") 
        return Model, added_reactions

This notebook will be used for testing the homology_gafilling() function as well as its following options:
- One or more templates
- Changing model/template objective
- Using all templates
- Changing the integer threshold
- Adding exchange reactions
- Adding transport reactions (3 different ways)

### Models


#### *E. coli*

In [2]:
# E. coli core
# bigg.ucsd.edu/models/e_coli_core
ECC = read_sbml_model("BiGG_files/e_coli_core.xml")
ECC

Using license file /home/fco/gurobi.lic
Academic license - for non-commercial use only - expires 2021-06-01


Name,e_coli_core
Memory address,0x07fac6ccd0040
Number of metabolites,72
Number of reactions,95
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


In [3]:
# E. coli str. C
# bigg.ucsd.edu/models/iEC1344_C
iEC1344_C = read_sbml_model("BiGG_files/iEC1344_C.xml")
iEC1344_C

Name,iEC1344_C
Memory address,0x07fac54c5a7f0
Number of metabolites,1934
Number of reactions,2726
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, periplasm, extracellular space"


In [4]:
# E. coli str. W
# bigg.ucsd.edu/models/iEC1364_W
iEC1364_W = read_sbml_model("BiGG_files/iEC1364_W.xml")
iEC1364_W

Name,iEC1364_W
Memory address,0x07fac6ccd47f0
Number of metabolites,1927
Number of reactions,2764
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


#### *P. putida*

In [5]:
# P. putida str. KT2440 
# bigg.ucsd.edu/models/iJN746
iJN746 = read_sbml_model("BiGG_files/iJN746.xml")
iJN746

Name,iJN746
Memory address,0x07fac6ccd49d0
Number of metabolites,907
Number of reactions,1054
Number of groups,0
Objective expression,1.0*BIOMASS_KT_TEMP - 1.0*BIOMASS_KT_TEMP_reverse_d18f7
Compartments,"extracellular space, cytosol, periplasm"


In [6]:
# P. putida str. KT2440 (full)
# bigg.ucsd.edu/models/iJN1463
iJN1463 = read_sbml_model("BiGG_files/iJN1463.xml")
iJN1463

Name,iJN1463
Memory address,0x07fabea66adf0
Number of metabolites,2153
Number of reactions,2927
Number of groups,0
Objective expression,1.0*BIOMASS_KT2440_WT3 - 1.0*BIOMASS_KT2440_WT3_reverse_d86d5
Compartments,"cytosol, extracellular space, periplasm"


#### *C. reinhardtii*

In [7]:
# C. reinhardtii
# bigg.ucsd.edu/models/iRC1080
iRC1080 = read_sbml_model("BiGG_files/iRC1080.xml")
iRC1080

No objective coefficients in model. Unclear what should be optimized


Name,iRC1080
Memory address,0x07fabea795100
Number of metabolites,1706
Number of reactions,2191
Number of groups,0
Objective expression,0
Compartments,"cytosol, mitochondria, chloroplast, flagellum, peroxisome/glyoxysome, nucleus, golgi apparatus, extracellular space, eyespot, thylakoid"


#### *B. subtilis*

In [8]:
# B. subtilis subsp. subtilis str. 168
# bigg.ucsd.edu/models/iYO844
iY0844 = read_sbml_model("BiGG_files/iYO844.xml")
iY0844

Name,iYO844
Memory address,0x07fabe8c8abe0
Number of metabolites,990
Number of reactions,1250
Number of groups,0
Objective expression,1.0*BIOMASS_BS_10 - 1.0*BIOMASS_BS_10_reverse_8788b
Compartments,"cytosol, extracellular space"


#### *S. cerevisiae*

In [9]:
# S. cerevisiae S288C
# bigg.ucsd.edu/models/iMM904
iMM904 = read_sbml_model("BiGG_files/iMM904.xml")
iMM904

Name,iMM904
Memory address,0x07fabe7a3e610
Number of metabolites,1226
Number of reactions,1577
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


In [10]:
# S. cerevisiae S288C (another version)
# bigg.ucsd.edu/models/iND750
iND750 = read_sbml_model("BiGG_files/iND750.xml")
iND750

Name,iND750
Memory address,0x07fabe72e8be0
Number of metabolites,1059
Number of reactions,1266
Number of groups,0
Objective expression,1.0*BIOMASS_SC4_bal - 1.0*BIOMASS_SC4_bal_reverse_bb385
Compartments,"extracellular space, cytosol, mitochondria, peroxisome/glyoxysome, nucleus, golgi apparatus, vacuole, endoplasmic reticulum"


#### *M. musculus*

In [11]:
# M. musculus
# bigg.ucsd.edu/models/iMM1415
iMM1415 = read_sbml_model("BiGG_files/iMM1415.xml")
iMM1415

No objective coefficients in model. Unclear what should be optimized


Name,iMM1415
Memory address,0x07fabe58daac0
Number of metabolites,2775
Number of reactions,3726
Number of groups,0
Objective expression,0
Compartments,"cytosol, extracellular space, golgi apparatus, lysosome, mitochondria, nucleus, endoplasmic reticulum, peroxisome/glyoxysome"


### Gap filling with default settings
- integer_threshold = 1e-06
- add_transport = False
- add_exchange = False

#### Filling a simple model with itself
E. coli core model will be used as it contains just 95 reactions.

First, we need to remove some reactions from the model. At the same time we will create another model with those reactions.

In [29]:
with suppress_stdout():
    x = read_sbml_model("BiGG_files/e_coli_core.xml")
x

Name,e_coli_core
Memory address,0x07fabc2e23700
Number of metabolites,72
Number of reactions,95
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


In [30]:
x.optimize().objective_value

0.8739215069684302

In [31]:
# Removing all reactions involving essential metabolite glc__D_e
y = cobra.Model("glc__D_e reactions")
for i in [i.id for i in x.metabolites.glc__D_e.reactions]:
    reaction = x.reactions.get_by_id(i)
    y.add_reaction(reaction.copy())
    x.remove_reactions([reaction])
y.reactions

[<Reaction GLCpts at 0x7fabc2afd6d0>, <Reaction EX_glc__D_e at 0x7fabc2799370>]

Calculating flux of the model:

In [32]:
print(x.optimize().objective_value)

None


Optimization is infeasible because we have deleted essential reactions. Carrying out gap filling:

In [33]:
with suppress_stdout():
    x_gp, added_reactions = homology_gapfilling(x, [y])

In [34]:
x_gp.optimize().objective_value

0.8739215069684302

Let's see the added reactions and some information about them.

In [35]:
pd.DataFrame(added_reactions)

,glc__D_e reactions
0,"(GLCpts, [b1621, b1817, b2416, b2415, b1818, b..."
1,"(EX_glc__D_e, Exchange reaction)"


- The title of the table shows the template name
- Each line represents an added reaction
- Each reaction is represented as a tuple with reaction name (left) and involved genes (right)

Now we will create two templates with these reactions.

In [36]:
with suppress_stdout():
    x = read_sbml_model("BiGG_files/e_coli_core.xml")

In [37]:
y = cobra.Model("GLCpts")
y.add_reaction(x.reactions.GLCpts.copy())
x.remove_reactions(x.reactions.GLCpts)

/home/fco/miniconda3/envs/TFM/lib/python3.8/site-packages/cobra/core/model.py:746: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [38]:
z = cobra.Model("EX_glc__D_e")
z.add_reaction(x.reactions.EX_glc__D_e.copy())
x.remove_reactions(x.reactions.EX_glc__D_e)

In [39]:
print(y.reactions, z.reactions)

[<Reaction GLCpts at 0x7fabc24ceac0>] [<Reaction EX_glc__D_e at 0x7fabc230b220>]


In [40]:
x.optimize().objective_value

/home/fco/miniconda3/envs/TFM/lib/python3.8/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


In [41]:
with suppress_stdout():
    X, added_reactions = homology_gapfilling(x, [y,z])

In [42]:
X

Name,e_coli_core
Memory address,0x07fabc264ebe0
Number of metabolites,72
Number of reactions,93
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


It appears that if several essential reactions (required for the model to work) are into different templates, they won't be added. That is reasonable if we take account of the algorithm's process.

#### Filling a more complex model with itself
A *S. cerevisiae* model with 1266 reactions will be used.

In [43]:
with suppress_stdout():
    x = read_sbml_model("BiGG_files/iND750.xml")

In [44]:
x

Name,iND750
Memory address,0x07fabc20d6670
Number of metabolites,1059
Number of reactions,1266
Number of groups,0
Objective expression,1.0*BIOMASS_SC4_bal - 1.0*BIOMASS_SC4_bal_reverse_bb385
Compartments,"extracellular space, cytosol, mitochondria, peroxisome/glyoxysome, nucleus, golgi apparatus, vacuole, endoplasmic reticulum"


Let's take a look into the objective reaction (biomass) in order not to remove it accidentally.

In [45]:
x.reactions.BIOMASS_SC4_bal.metabolites

{<Metabolite 13BDglcn_c at 0x7fabc1facd30>: -1.1348,
 <Metabolite ala__L_c at 0x7fabc1fc05b0>: -0.4588,
 <Metabolite amp_c at 0x7fabc20147c0>: -0.046,
 <Metabolite arg__L_c at 0x7fabc1fed640>: -0.1607,
 <Metabolite asn__L_c at 0x7fabc1fff820>: -0.1017,
 <Metabolite asp__L_c at 0x7fabc2039730>: -0.2975,
 <Metabolite atp_c at 0x7fabc1fff430>: -59.276,
 <Metabolite cmp_c at 0x7fabc1f6eaf0>: -0.0447,
 <Metabolite cys__L_c at 0x7fabc1fa0790>: -0.0066,
 <Metabolite damp_c at 0x7fabc1f381c0>: -0.0036,
 <Metabolite dcmp_c at 0x7fabc1f38d60>: -0.0024,
 <Metabolite dgmp_c at 0x7fabc1f64700>: -0.0024,
 <Metabolite dtmp_c at 0x7fabc1eb1bb0>: -0.0036,
 <Metabolite ergst_c at 0x7fabc1ec51f0>: -0.0007,
 <Metabolite gln__L_c at 0x7fabc207ccd0>: -0.1054,
 <Metabolite glu__L_c at 0x7fabc207ceb0>: -0.3018,
 <Metabolite gly_c at 0x7fabc1fac460>: -0.2904,
 <Metabolite glycogen_c at 0x7fabc1ec5fd0>: -0.5185,
 <Metabolite gmp_c at 0x7fabc1ed82b0>: -0.046,
 <Metabolite h2o_c at 0x7fabc1fedca0>: -59.276,
 <Met

In [46]:
x.optimize().objective_value

0.09732337590376772

In [47]:
y = cobra.Model("co2_c reactions")
for i in [i.id for i in x.metabolites.co2_c.reactions]:
    reaction = x.reactions.get_by_id(i)
    y.add_reaction(reaction.copy())
    x.remove_reactions([reaction])
y

Name,co2_c reactions
Memory address,0x07fabc20d6c10
Number of metabolites,102
Number of reactions,55
Number of groups,0
Objective expression,0
Compartments,"c, m, x, e, v, n, g"


55 reactions have been taken from the model.

In [48]:
x.optimize().objective_value

0.0

In [49]:
len(x.reactions)

1211

In [50]:
with suppress_stdout():
    X, added_reactions = homology_gapfilling(x, [y])

In [51]:
X.optimize().objective_value

0.0

In [52]:
len(X.reactions)

1211

Adding exchange and transport reactions (we will get into this options later):

In [53]:
with suppress_stdout():
    X, added_reactions = homology_gapfilling(x, [y], force_transport=True, force_exchange=True)

In [54]:
X.optimize().objective_value

0.0

In [55]:
len(X.reactions)

1217

Lowering integrality threshold:

In [56]:
X, added_reactions = homology_gapfilling(x, [y], force_transport=True, force_exchange=True, integer_threshold=1e-9)

Read LP format model from file /tmp/tmpwdq2y3e8.lp
Reading time = 0.01 seconds
: 1059 rows, 2434 columns, 9984 nonzeros
Read LP format model from file /tmp/tmp20jdev77.lp
Reading time = 0.01 seconds
: 1059 rows, 2434 columns, 9984 nonzeros
Read LP format model from file /tmp/tmp7p70sgav.lp
Reading time = 0.00 seconds
: 102 rows, 110 columns, 640 nonzeros
Read LP format model from file /tmp/tmpn22ukiv2.lp
Reading time = 0.00 seconds
: 102 rows, 110 columns, 640 nonzeros

co2_c reactions: failed to validate gapfilled model, try lowering the integer_threshold


In [57]:
X.optimize().objective_value

0.0

In [58]:
len(X.reactions)

1217

In this case all the options seem not to be enough for the gap filling to be efficient. Also the algorithm demands to lower the integrality value but it can't be lowered anymore. This situation is just too complex for this algorithm. 

#### Filling a model with another
We will try to improve the biomass production of *E. coli* core model using other templates.

In [59]:
with suppress_stdout():
    x = read_sbml_model("BiGG_files/e_coli_core.xml")

In [60]:
print(x.objective)

Maximize
1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5


In [61]:
x.optimize().objective_value

0.8739215069684302

In [62]:
with suppress_stdout():
    X, added_reactions = homology_gapfilling(x, [iEC1344_C, iEC1364_W])

In [116]:
pd.DataFrame(added_reactions)

,iEC1344_C


No reaction has been added through gap filling. Probably the *E. coli* core model is optimized to carry out the best biomass production. It stopped after first template because they are supposed to be sorted by homology/identity with the model so if the first template didn't work the second one isn't expected to do so. Let's force the use of both templates just in case.

In [63]:
with suppress_stdout():
    X, added_reactions = homology_gapfilling(x, [iEC1344_C, iEC1364_W], use_all_templates = True)

In [64]:
pd.DataFrame(added_reactions)

,iEC1344_C,iEC1364_W


It didn't add any reaction neither. Now we will use all available templates.

In [65]:
with suppress_stdout():
    x = read_sbml_model("BiGG_files/e_coli_core.xml")

In [66]:
T = load_template_models([iEC1344_C, iEC1364_W, iND750, iJN1463, iJN746, iMM904, iMM1415, iY0844])

In [67]:
T

[<Model iEC1344_C at 0x7fac54c5a7f0>,
 <Model iEC1364_W at 0x7fac6ccd47f0>,
 <Model iND750 at 0x7fabe72e8be0>,
 <Model iJN1463 at 0x7fabea66adf0>,
 <Model iJN746 at 0x7fac6ccd49d0>,
 <Model iMM904 at 0x7fabe7a3e610>,
 <Model iMM1415 at 0x7fabe58daac0>,
 <Model iYO844 at 0x7fabe8c8abe0>]

In [68]:
with suppress_stdout():
    X, added_reactions = homology_gapfilling(x, T, use_all_templates=True)

In [69]:
pd.DataFrame(added_reactions)

,iEC1344_C,iEC1364_W,iND750,iJN1463,iMM904,iMM1415,iYO844


The only model which has not been able to use is iJN746.
Changing model's objective:

In [70]:
with suppress_stdout():
    X, added_reactions = homology_gapfilling(x, T, use_all_templates=True, model_obj="CO2t")

In [71]:
X.optimize().objective_value

11.104242424242424

In [72]:
pd.DataFrame(added_reactions)

,iEC1344_C,iEC1364_W,iND750,iJN1463,iMM904,iMM1415,iYO844


The objective has been successfully changed but no reaction has been added.

Next we will try the same but using a complex model as query.

In [87]:
with suppress_stdout():
    x = read_sbml_model("BiGG_files/iEC1364_W.xml")

In [88]:
print(x.objective, x.optimize().objective_value)

Maximize
1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1 0.985364865731541


In [89]:
T = [iEC1344_C, iJN746, iMM904, iY0844]

In [90]:
with suppress_stdout():
    X, added_reactions = homology_gapfilling(x, T, use_all_templates=False)

In [91]:
X.optimize().objective_value

0.985364865731541

In [92]:
pd.DataFrame(added_reactions)

,iEC1344_C


In [93]:
# forcing the use of all templates
with suppress_stdout():
    X, added_reactions = homology_gapfilling(x, T, use_all_templates=True)

In [94]:
X.optimize().objective_value

0.1876164009738142

In [95]:
added_reactions

{'iEC1344_C': [],
 'iJN746': [('BIOMASS_KT_TEMP', [])],
 'iMM904': [],
 'iYO844': []}

In [96]:
X.reactions.BIOMASS_KT_TEMP

Reaction identifier,BIOMASS_KT_TEMP
Name,BiomassKT TEMP
Memory address,0x07fab9f1e0cd0
Stoichiometry,0.05 5mthf_c + 5e-05 accoa_c + 0.488 ala__L_c + 0.001 amp_c + 0.281 arg__L_c + 0.229 asn__L_c + 0.229 asp__L_c + 45.7318 atp_c + 6e-06 coa_c + 0.0005 cpe160_c + 0.0005 cpe180_c + 0.0005 cpg160_c + ... 0.05 5-Methyltetrahydrofolate + 5e-05 Acetyl-CoA + 0.488 L-Alanine + 0.001 AMP C10H12N5O7P + 0.281 L-Arginine + 0.229 L-Asparagine + 0.229 L-Aspartate + 45.7318 ATP C10H12N5O13P3 + 6e-06 Coenzyme A...
GPR,
Lower bound,1.0
Upper bound,1.4


This is a strange behavior. The algorithm has filled the model with a reaction which lowers the objective value. If that would have happened with use_all_templates = False (default), the bounds of the added reaction would be set to 0. 